<a href="https://colab.research.google.com/github/darknight11345/turtlebot4/blob/main/Text25_01_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!nvidia-smi

Sat Jan 25 18:51:40 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
pwd

'/content'

In [3]:
#setup of dataset
import torch
print (torch.cuda.is_available())

True


In [4]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 116.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.5 MB/s eta 0:00:00


In [5]:
!pip install fsspec==2024.10.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.2.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.10.0 which is incompatible.


In [34]:
import pandas as pd
data = pd.read_csv("ner_dataset_with_head_towards_and_noise_23_01_25.csv",encoding="latin1", delimiter= ";" )

In [35]:
data.head(30)

,Sentence_ID,Word,POS,NER
0,1,Deliver,VERB,ACTION
1,1,Penicillin,PROPN,O
2,1,to,ADP,O
3,1,ICU,PROPN,LOC
4,2,Move,VERB,ACTION
5,2,to,ADP,O
6,2,Reception,PROPN,LOC
7,2,with,ADP,O
8,2,Penicillin,PROPN,O
9,3,Proceed,VERB,ACTION


In [36]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [37]:
data.rename(columns={"Sentence_ID":"sentence_id","Word":"words","NER":"labels"}, inplace =True)

In [38]:
data["labels"] = data["labels"].str.upper()
data["words"] = data["words"].str.lower()

In [39]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [40]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [41]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [42]:
test_data

,sentence_id,words,labels
4748,949,bring,ACTION
5727,1143,paracetamol,O
7843,1569,with,O
4993,997,room,BEG-LOC
4975,994,ibuprofen,O
...,...,...,...
1750,348,towards,O
6296,1260,proceed,ACTION
6491,1299,ibuprofen,O
5356,1069,with,O


In [43]:
train_data

,sentence_id,words,labels
8452,1693,reception,LOC
9769,1952,with,O
9814,1961,230,INS-LOC
9703,1939,to,O
5847,1167,to,O
...,...,...,...
1950,388,aspirin,O
1066,209,with,O
7156,1432,ibuprofen,O
8742,1750,517,INS-LOC


In [ ]:
#Model Training

In [16]:

!pip install torch  --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124


In [17]:


from simpletransformers.ner import NERModel,NERArgs

In [44]:
label = data["labels"].unique().tolist()
label

['ACTION', 'O', 'LOC', 'BEG-LOC', 'INS-LOC']

In [45]:
args = NERArgs()
args.num_train_epochs = 11
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32
args.gradient_accumulation_steps = 4  # Adjust this to your needs
args.max_seq_length = 128  # Adjust this according to your data
args.evaluate_during_training: True


In [47]:
model = NERModel('bert', 'bert-base-uncased',labels=label,args =args, use_cuda=True)
#model = NERModel("bert", "outputs", use_cuda=False) # to use pretained model

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
#model.train_model(train_data,eval_data = test_data,acc=accuracy_score)
output_dir = "./best_model"
#output_dir = "./best_model_pretained_model"
model.train_model(train_data, eval_data=test_data, acc=accuracy_score, save_best_model=True, output_dir=output_dir)

model.save_model("./text_trained_model_bert")

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/11 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:758: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 11:   0%|          | 0/63 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:782: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


...

Running Epoch 7 of 11:   0%|          | 0/63 [00:00<?, ?it/s]

Running Epoch 8 of 11:   0%|          | 0/63 [00:00<?, ?it/s]

Running Epoch 9 of 11:   0%|          | 0/63 [00:00<?, ?it/s]

Running Epoch 10 of 11:   0%|          | 0/63 [00:00<?, ?it/s]

Running Epoch 11 of 11:   0%|          | 0/63 [00:00<?, ?it/s]

In [49]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/42 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:1303: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ACTION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: INS-LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: BEG-LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [50]:
result

{'eval_loss': 0.04083378566429019,
 'precision': 0.9869739478957916,
 'recall': 0.9969635627530364,
 'f1_score': 0.9919436052366565}

In [55]:
prediction, model_output = model.predict(["    to  111 room deliver tablets "])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [56]:
prediction

[[{'deliver': 'ACTION'},
  {'to': 'O'},
  {'111': 'INS-LOC'},
  {'room': 'BEG-LOC'},
  {'tablets': 'O'}]]

In [57]:
model_output

[[{'deliver': [[6.93, 1.595, -2.45, -2.81, -2.016]]},
  {'to': [[0.1304, 7.4, -1.964, -2.395, -1.851]]},
  {'111': [[-2.041, -0.2893, -2.023, -1.87, 6.336]]},
  {'room': [[-2.629, 0.764, -2.89, 5.945, -1.735]]},
  {'tablets': [[-0.9473, 7.504, -1.865, -1.859, -1.738]]}]]

In [ ]:
#inference

# Load the best saved model for inference
#model1 = NERModel("bert", "./best_model", use_cuda=False)


In [63]:
#model1 = NERModel("bert", "./outputs", use_cuda=False)

model1 = NERModel("bert", "./best_model/checkpoint-165-epoch-11", use_cuda=True)

In [93]:

text="send  tablet to room number 00111"
text=text.lower()
prediction1, model_output = model1.predict([text])


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [94]:
prediction1

[[{'send': 'ACTION'},
  {'tablet': 'O'},
  {'to': 'O'},
  {'room': 'BEG-LOC'},
  {'number': 'O'},
  {'00111': 'INS-LOC'}]]

provide

In [85]:
# Access the first (inner) list
inner_list = prediction1[0]


# Extract entries where the label contains 'LOC'
locations = [
    {key} for item in inner_list for key, value in item.items() if "LOC" in value
]

Action =[
{key} for action in inner_list for key,value in action.items() if "ACTION" in value
]
print("Locations:", locations)
print("Action:",Action)






Locations: [{'111'}, {'room'}]
Action: [{'deliver'}]
